In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import log, square, sqrt, power, arange, ones, zeros, isscalar,\
    array, outer, pi, sin, cos, expand_dims, repeat, full, concatenate, ravel
from numpy.random import normal
from scipy.optimize import least_squares, minimize
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from Module.baseModule.bayesFlexibleFourier import *
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [23]:
# functions for loading the datasets
def load_stock():
    df_stock = pd.read_csv('NSQ_OneYear100test_Sept21.csv', 
                       usecols=['Local_Date_Time','RIC','Open','High','Low','Close','VWAP','NumberOfTrades','Volume'],
                       dtype={'Local_Date_Time': str,
                              'RIC': str, 
                              'Open': np.float64,
                              'High': np.float64,
                              'Low': np.float64,
                              'Close': np.float64,
                              'VWAP': np.float64,
                              'NumberOfTrades': int,
                              'Volume': int},
                       skipinitialspace=True,
                       parse_dates=True)
    return df_stock

def load_stockqqq():
    df_stockqqq = pd.read_csv('QQQ_OneYear100test_Sept21.csv', 
                   usecols=['Local_Date_Time','RIC','Open','High','Low','Close','VWAP','NumberOfTrades','Volume'],
                   dtype={'Local_Date_Time': str,
                          'RIC': str, 
                          'Open': np.float64,
                          'High': np.float64,
                          'Low': np.float64,
                          'Close': np.float64,
                          'VWAP': np.float64,
                          'NumberOfTrades': int,
                          'Volume': int},
                   skipinitialspace=True,
                   parse_dates=True)
    return df_stockqqq

def load_auction():
    df_auction = pd.read_csv('NSQ_OneYear100closeA_Sept21.csv',
                          skipinitialspace=True,
                          parse_dates=True)
    return df_auction

def load_auctionqqq():
    df_auctionqqq = pd.read_csv('QQQ_OneYear100closeA_Sept21.csv',
                      skipinitialspace=True,
                      parse_dates=True)
    return df_auctionqqq

In [24]:
# Trade data
def load_stockdata(df_stock, df_stockqqq, stock_name):
    #NSQ_OneYear100closeA_Sept21.csv
    df_stockRIC = df_stock[df_stock['RIC'] == stock_name]
    df_stockRIC['Date'] = pd.to_datetime(df_stockRIC['Local_Date_Time']).dt.date
    df_stockqqq['Date'] = pd.to_datetime(df_stockqqq['Local_Date_Time']).dt.date
    
    return df_stockRIC, df_stockqqq

In [25]:
# Auction price data
def load_auctiondata(df_auction, df_auctionqqq, stock_name):
    df_auctionRIC = df_auction[df_auction['RIC'] == stock_name]
    df_auctionRIC['Date'] = pd.to_datetime(df_auctionRIC['Local_Date_Time']).dt.date
    df_auctionqqq['Date'] = pd.to_datetime(df_auctionqqq['Local_Date_Time']).dt.date
    
    return df_auctionRIC, df_auctionqqq

In [26]:
# Data Processing (10 minute intervals)

def get_voldf(df_stockRIC, df_auctionRIC):
    dates = df_stockRIC['Date'].unique()
    dates_final = []
    stocks = []
    daily_return = []
    auction_log_returns = []
    auction_log_returns_340350 = []
    daily_volatility = []
    daily_volatility_minus4pm = []
    avg_20day_volatility = []

    for date in dates:

        stock_today = df_stockRIC[df_stockRIC['Date'] == date]
        df_auctionRIC_today = df_auctionRIC[df_auctionRIC['Date'] == date]
        stock_today = stock_today.set_index(pd.DatetimeIndex(stock_today['Local_Date_Time']))
        stock_today_10min_max = pd.DataFrame(stock_today['High'].resample("10T").max())
        stock_today_10min_min = pd.DataFrame(stock_today['Low'].resample("10T").min())
        stock_today_10min_open = pd.DataFrame(stock_today['Open'].resample("10T").first())
        stock_today_10min_close = pd.DataFrame(stock_today['Close'].resample("10T").last())
        stock_today_10min = stock_today_10min_max.join(stock_today_10min_min)
        stock_today_10min = stock_today_10min.join(stock_today_10min_open)
        stock_today_10min = stock_today_10min.join(stock_today_10min_close)

        # Garman and Klass Volatility formula
        stock_today_10min['Volatility'] = np.sqrt(0.5*np.square(np.log(stock_today_10min['High']/stock_today_10min['Low'])) - (2*np.log(2)-1)*np.square(np.log(stock_today_10min['Close']/stock_today_10min['Open'])))
        #stock_today_10min['log_returns'] = abs(np.log(stock_today_10min['Close']/stock_today_10min['Open'])) # alternate volatility formula

        vol_today = [val for val in stock_today_10min['Volatility'].values]
        return_today = (stock_today_10min['Close']/stock_today_10min['Open']).values

        if len(vol_today) != 39: # Skip if there are fewer than 390 minutes of trading data
            continue

        # Dates for which the trade data is complete
        dates_final.append(date)

        # Change the 39th 10min interval to include the auction price (instead of close)
        auction_volatility = np.sqrt(0.5*np.square(np.log(stock_today_10min['High'][-1]/stock_today_10min['Low'][-1])) - (2*np.log(2)-1)*np.square(np.log(df_auctionRIC_today['Price'].iloc[0]/stock_today_10min['Open'][-1])))
        vol_today[-1] = auction_volatility

        # 10 minute interval volatility * number of complete trade days
        stocks.append(vol_today)

        # Daily raw return
        daily_return.append(return_today)

        # Auction Volatility Information
        auction_log_returns_today = abs(np.log(df_auctionRIC_today['Price'].iloc[0]/stock_today_10min['Open'][-1]))
        auction_log_returns.append(auction_log_returns_today)

        # Naive Auction Volatility Estimate (Volatility between 3:40pm and 3:50pm)
        auction_log_returns_today_340350 = abs(np.log(stock_today_10min['Open'][-1]/stock_today_10min['Open'][-2]))
        auction_log_returns_340350.append(auction_log_returns_today_340350)

        # Average daily volatility (later used for EWMA 20Day)
        daily_volatility.append(sum(vol_today))
        daily_volatility_minus4pm.append(sum(vol_today[:-1]))

    daily_return = np.array(daily_return)
    vol_df = pd.DataFrame({'Date':dates_final, 'daily_volatility': daily_volatility, 'daily_volatility_minus4pm': daily_volatility_minus4pm})
    vol_df['volatility_ewma20'] = np.array([None] + [i for i in vol_df['daily_volatility'].ewm(span=20).mean()][:-1])

    stocks = stocks[20:]
    stocks = np.vstack(stocks).T # Training data for Anderson model
    avg_20day_volatility_raw = np.array(vol_df['volatility_ewma20'])
    avg_20day_volatility = np.array(vol_df['volatility_ewma20'][20:]) # Sigma_t input for Anderson model
    
    return vol_df


In [27]:
def mod1B_results(vol_df):
    vol_df = vol_df
    vol_df['auction'] = vol_df['daily_volatility'] - vol_df['daily_volatility_minus4pm']
    vol_df['volatility_ewma20_minus4pm'] = np.array([None] + [i for i in vol_df['daily_volatility_minus4pm'].ewm(span=20).mean()][:-1])
    vol_df['volatility_ewma20_auction'] = np.array([None] + [i for i in vol_df['auction'].ewm(span=20).mean()][:-1])
    
    df_mod1B = pd.DataFrame()
    df_mod1B['Date'] = vol_df['Date'][20:]
    df_mod1B['Actual Volatility'] = vol_df['auction'][20:]
    df_mod1B['Predicted Volatility'] = vol_df['volatility_ewma20_auction'][20:]
    
    MAE = mean_absolute_error(df_mod1B['Actual Volatility'][-211:], df_mod1B['Predicted Volatility'][-211:])
    MSE = mean_squared_error(df_mod1B['Actual Volatility'][-211:], df_mod1B['Predicted Volatility'][-211:])
    R2 = r2_score(df_mod1B['Actual Volatility'][-211:], df_mod1B['Predicted Volatility'][-211:])
    
    return MAE, MSE, R2

In [36]:
def mod1C_results(vol_df):
    vol_df = vol_df
    vol_df['volatility_ewma20_minus4pm_yest'] = vol_df['volatility_ewma20_minus4pm'].shift(1)
    vol_df['volatility_ewma20_auction_1c'] = vol_df['daily_volatility_minus4pm']/vol_df['volatility_ewma20_minus4pm_yest']*vol_df['volatility_ewma20_auction']
    
    df_mod1C = pd.DataFrame()
    df_mod1C['Date'] = vol_df['Date'][20:]
    df_mod1C['Actual Volatility'] = vol_df['auction'][20:]
    df_mod1C['Predicted Volatility'] = vol_df['volatility_ewma20_auction_1c'][20:]
    
    MAE = mean_absolute_error(df_mod1C['Actual Volatility'][-211:], df_mod1C['Predicted Volatility'][-211:])
    MSE = mean_squared_error(df_mod1C['Actual Volatility'][-211:], df_mod1C['Predicted Volatility'][-211:])
    R2 = r2_score(df_mod1C['Actual Volatility'][-211:], df_mod1C['Predicted Volatility'][-211:])

    return MAE, MSE, R2


In [37]:
########### FOURIER MOVING WINDOW (20Days) ###########
def process_data_moving_window(df_stockRIC, df_auctionRIC):
    dates = df_stockRIC['Date'].unique()
    dates_final = []
    stocks = []
    daily_return = []
    auction_log_returns = []
    auction_log_returns_340350 = []
    daily_volatility = []
    daily_volatility_minus4pm = []
    avg_20day_volatility = []

    for date in dates:
        
        stock_today = df_stockRIC[df_stockRIC['Date'] == date]
        df_auctionRIC_today = df_auctionRIC[df_auctionRIC['Date'] == date]
        stock_today = stock_today.set_index(pd.DatetimeIndex(stock_today['Local_Date_Time']))
        stock_today_10min_max = pd.DataFrame(stock_today['High'].resample("10T").max())
        stock_today_10min_min = pd.DataFrame(stock_today['Low'].resample("10T").min())
        stock_today_10min_open = pd.DataFrame(stock_today['Open'].resample("10T").first())
        stock_today_10min_close = pd.DataFrame(stock_today['Close'].resample("10T").last())
        stock_today_10min = stock_today_10min_max.join(stock_today_10min_min)
        stock_today_10min = stock_today_10min.join(stock_today_10min_open)
        stock_today_10min = stock_today_10min.join(stock_today_10min_close)
        
        # Garman and Klass Volatility formula
        stock_today_10min['Volatility'] = np.sqrt(0.5*np.square(np.log(stock_today_10min['High']/stock_today_10min['Low'])) - (2*np.log(2)-1)*np.square(np.log(stock_today_10min['Close']/stock_today_10min['Open'])))   
        vol_today = [val for val in stock_today_10min['Volatility'].values]
        
        if len(vol_today) != 39: # Skip if there are fewer than 390 minutes of trading data
            continue
            
        # Dates for which the trade data is complete
        dates_final.append(date)
        
        # Change the 39th 10min interval to include the auction price (instead of close)
        auction_volatility = np.sqrt(0.5*np.square(np.log(stock_today_10min['High'][-1]/stock_today_10min['Low'][-1])) - (2*np.log(2)-1)*np.square(np.log(df_auctionRIC_today['Price'].iloc[0]/stock_today_10min['Open'][-1])))
        vol_today[-1] = auction_volatility
        
        # 10 minute interval volatility * number of complete trade days
        stocks.append(vol_today)
            
        # Auction Log Return Information
        auction_log_returns_today = abs(np.log(df_auctionRIC_today['Price'].iloc[0]/stock_today_10min['Open'][-1]))
        auction_log_returns.append(auction_log_returns_today)
        
        # Naive Auction Log Return Estimate (Volatility between 3:40pm and 3:50pm)
        auction_log_returns_today_340350 = abs(np.log(stock_today_10min['Open'][-1]/stock_today_10min['Open'][-2]))
        auction_log_returns_340350.append(auction_log_returns_today_340350)
        
        # Average daily volatility (later used for EWMA 20Day)
        daily_volatility.append(sum(vol_today))
        daily_volatility_minus4pm.append(sum(vol_today[:-1]))

    # daily_return = np.array(daily_return)
    vol_df = pd.DataFrame({'Date':dates_final, 'daily_volatility': daily_volatility, 'daily_volatility_minus4pm': daily_volatility_minus4pm})
    vol_df['volatility_ewma20'] = np.array([None] + [i for i in vol_df['daily_volatility'].ewm(span=20).mean()][:-1])

    stocks = np.vstack(stocks)[20:].T # Training data for Anderson model
    avg_20day_volatility = np.array(vol_df['volatility_ewma20'])[20:]

    return stocks, avg_20day_volatility


In [38]:
# fourier 2A - Sliding window 
def fourier2A_results(stocks, avg_20day_volatility, J, P):
    actual_fourier_window = []
    predicted_fourier_window = []
    predicted_fourier_window_before4pm = []
    
    for i in range(211):
        
        if i % 50 == 0:
            print(i/210, "done")
            
        stocks_window = stocks[:,i:i+20]
        avg_20day_volatility_window = avg_20day_volatility[i:i+20]
        fourier_window = flexible_fourier_regression(N=39, di=[], J=J, P=P) # 39 10-minute intervals in each trade day
        res_window = fourier_window.train(stocks_window, avg_20day_volatility_window, 0.0000005)
        #print("optimizer success: {}".format(res_window.success))
        #print("objective function (mse): {:.8f}".format(res_window.fun))
        
        # No Bayes
        result = fourier_window.predict(avg_20day_volatility[i+20])
        
        actual_fourier_window.append(stocks.T[i+20][-1])
        predicted_fourier_window.append(result[-1][0])
        predicted_fourier_window_before4pm.append(sum([res[0] for res in result[:-1]]))
        #print(i)

    MAE = mean_absolute_error(actual_fourier_window, predicted_fourier_window)
    MSE = mean_squared_error(actual_fourier_window, predicted_fourier_window)
    R2 =r2_score(actual_fourier_window, predicted_fourier_window)
    
    return MAE, MSE, R2, fourier_window, predicted_fourier_window, predicted_fourier_window_before4pm


In [39]:
# fourier 2B - Sliding window - Adaptive
def fourier2B_results(stocks, avg_20day_volatility, fourier_window, J, P):
    actual_fourier_window = []
    predicted_fourier_window = []

    for i in range(211):
        # After Bayes
        #print(i)
        
        bayes_dayVol = fourier_window.vol_update(stocks.T[i+20,:39], avg_20day_volatility[i+20], tol=1e-8)
        result = fourier_window.predict(bayes_dayVol)
        
        actual_fourier_window.append(stocks.T[i+20][-1])
        predicted_fourier_window.append(result[-1][0])
        
        
    MAE = mean_absolute_error(actual_fourier_window, predicted_fourier_window)
    MSE = mean_squared_error(actual_fourier_window, predicted_fourier_window)
    R2 =r2_score(actual_fourier_window, predicted_fourier_window)

    return MAE, MSE, R2, actual_fourier_window

In [40]:
# fourier 2C - Sliding window - Adaptive
def fourier2C_results(vol_df, actual_fourier_window, predicted_fourier_window, predicted_fourier_window_before4pm):
    vol_df_window = vol_df[40:]
    vol_df_window['volatility_fourier_window'] = predicted_fourier_window
    vol_df_window['predicted_fourier_window_before4pm_yest'] = np.array([None] + predicted_fourier_window_before4pm[:-1])
    vol_df_window['volatility_fourier_2c'] = vol_df_window['daily_volatility_minus4pm']/vol_df_window['predicted_fourier_window_before4pm_yest']*vol_df_window['volatility_fourier_window']
    predicted_fourier_window2C = np.array(vol_df_window['volatility_fourier_2c'])

    MAE = mean_absolute_error(actual_fourier_window[1:], predicted_fourier_window2C[1:])
    MSE = mean_squared_error(actual_fourier_window[1:], predicted_fourier_window2C[1:])
    R2 = r2_score(actual_fourier_window[1:], predicted_fourier_window2C[1:])

    return MAE, MSE, R2

In [41]:
# combine all functions 
def results(df_stock, df_stockqqq, df_auction, df_auctionqqq, stock_name, J=1, P=15):
    
    #print("part 1")
    df_stockRIC, df_stockqqq = load_stockdata(df_stock, df_stockqqq, stock_name)
    df_auctionRIC, df_auctionqqq = load_auctiondata(df_auction, df_auctionqqq, stock_name)
    vol_df = get_voldf(df_stockRIC, df_auctionRIC)
    
    #print("part 2")
    MAE_1b, MSE_1b, R2_1b = mod1B_results(vol_df)
    MAE_1c, MSE_1c, R2_1c = mod1C_results(vol_df)
    
    #print("part 3")
    stocks, avg_20day_volatility = process_data_moving_window(df_stockRIC, df_auctionRIC)
    MAE_2a, MSE_2a, R2_2a, fourier_window, predicted_fourier_window, predicted_fourier_window_before4pm = fourier2A_results(stocks, avg_20day_volatility, J, P)
    MAE_2b, MSE_2b, R2_2b, actual_fourier_window = fourier2B_results(stocks, avg_20day_volatility, fourier_window, J, P)
    MAE_2c, MSE_2c, R2_2c = fourier2C_results(vol_df, actual_fourier_window, predicted_fourier_window, predicted_fourier_window_before4pm)

    data = {
        "Metrics": ["EWMA Auction Vol", "EWMA Auction Vol/ratio",
                    "Fourier Auction Vol", "Fourier Auction Vol Adaptive", "Fourier Auction Vol/ratio"],
        "MAE": [MAE_1b, MAE_1c, MAE_2a, MAE_2b, MAE_2c],
        "MSE": [MSE_1b, MSE_1c, MSE_2a, MSE_2b, MSE_2c],
        "R^2": [R2_1b, R2_1c, R2_2a, R2_2b, R2_2c]
    }
    
    pd.set_option('display.float_format',
      lambda x: '{:,.4f}'.format(x) if abs(x) >0.01 else '{:,.3e}'.format(x))
    
    table = pd.DataFrame(data, index=None)
    print(stock_name)
    display(table)

# Run the code

In [34]:
# load the datasets
df_stock = load_stock()
df_stockqqq = load_stockqqq()
df_auction = load_auction()
df_auctionqqq = load_auctionqqq()

In [42]:
choose interested stocks
stock_list = ["MSFT.O", "AAPL.O", "GOOGL.O", "MRNA.O", "INTC.O", "PEP.O",
              "ZM.O", "EBAY.O", "PTON.O"]
hyperparameters = [(1,25), (1,25), (1,25), (1,25), (1,25), (1,25), (1,25), (1,25), (1,25)]

for i in range(9):
    results(df_stock, df_stockqqq, df_auction, df_auctionqqq, stock_list[i], J=hyperparameters[i][0], P=hyperparameters[i][1])
    

0.0 done
0.23809523809523808 done
0.47619047619047616 done
0.7142857142857143 done
0.9523809523809523 done
AAPL.O


,Metrics,MAE,MSE,R^2
0,EWMA Auction Vol (1B),9.671e-04,2.497e-06,0.1325
1,EWMA Auction Vol/ratio (1C),8.346e-04,2.197e-06,0.2368
2,Fourier Auction Vol (2A),1.094e-03,3.051e-06,-0.0600
3,Fourier Auction Vol Adaptive (2B),1.154e-03,3.789e-06,-0.3162
4,Fourier Auction Vol/ratio (2C),9.622e-04,2.572e-06,0.1057


IndexError: list index out of range